# New York City's Taxi Fare Dataset.

Performing Exploratory Data Analysis on NYC Taxi Fare Dataset.
https://www.kaggle.com/datasets/diishasiing/revenue-for-cab-drivers

## 1. Importing Necessary Libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 2. Importing the Dataset.

In [ ]:
data = pd.read_csv("TaxiFareDataset.csv")
data.head()

In [ ]:
data.tail()

Knowing the Dimensionality and Attribute Types.

In [ ]:
print(f'The Dimensionality of Dataset: {data.shape}')
print('\033[33mThe Attributes of Dataset and types\033[0m')
print(data.dtypes)

Characteristics of Numerical Dataset.

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
# Dimensions to represent the Data.
data.ndim

Unique Values and their count in the Dataset.

In [ ]:
cat_attr = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'store_and_fwd_flag']
for col in cat_attr:
    print(f'\033[33mNo. of Unique Values for {col}\033[0m : {len(data[col].unique())}')
    print(f'Unique Values : {data[col].unique()}')

From the Demographics of the Dataset we can see that there is a need for Data Preprocessing as: -  
1. There Exists **NULL Values** in the Dataset for certain records
2. There might be chance of Duplicate Records as well.
3. Certain Attributes like **trip_distance** has negative values where as distance is always a positive unit, **Amount**, **tip_amount**, etc., also contains negative value as minimum.


## 3. Handling Null Values.

### a. Identifying NULL Values.

In [ ]:
data.isnull().sum()

In [ ]:
# Alternative way to find NULL Values.

missing_data = data.isnull()

for col in missing_data.columns.values.tolist():
    print(f'\033[33m{col}\033[0m')
    print(missing_data[col].value_counts())
    print('')

Hence, the NULL Values are: 1. VendorID, 2. passenger_count, 3. RatecodeID, 4. store_and_fwd_flag, 5. payment_type.

### b. Filling up or Removing NULL Value Records.

In [ ]:
data['VendorID'].values

In [ ]:
# Analysing the Distribution of the VendorID Attribute.

plt.figure(figsize=(10, 5))
plt.title('Distribution of VendorID')
sns.histplot(data['VendorID'], kde=True, bins=10)
plt.show()

From the Data Distribution, and from the Dataset Description about **VendorID** Column, it describes the unique identifier for the taxi vendor or service provider. So, the records with null values in VendorID needs to be dropped as the future analysis comaprision between two types of providers is significant which gets changed if we replace the null values with value '2'.

In [ ]:
data = data.dropna(subset=['VendorID'])

In [ ]:
data.shape

In [ ]:
# Analysing the Distribution of the Attributes
cols = ['passenger_count', 'RatecodeID', 'store_and_fwd_flag', 'payment_type']

for col in cols:
    print(f'\033[33mThe Unique Values of the {col} are of\033[0m: {data[col].unique()}')

In [ ]:
# Ensuring categorical columns to string type for plotting
data['store_and_fwd_flag'] = data['store_and_fwd_flag'].astype(str)
data['payment_type'] = data['payment_type'].astype(str)

In [ ]:
# Plotting the Distributions of the Attributes

# Setting up subplots
plt.figure(figsize=(12, 8))

for i, col in enumerate(cols, 1):
    plt.subplot(2, 2, i)
    
    # Use histplot for numerical data, countplot for categorical
    if data[col].dtype in ['int64', 'float64']:
        sns.histplot(data[col], bins=20, kde=True)
    else:
        sns.countplot(x=data[col], order=data[col].value_counts().index)
    
    plt.title(f'Distribution of {col}')
    plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


Since the attributes **passenger_count**, are highly skewed data with continuous values from 0 to 9. we can choose median to fill the missing values than the mean type.

In [ ]:
data['passenger_count'].fillna(data['passenger_count'].median().astype(float), inplace=True)

Remaining the attributes **RatecodeID, Store_and_fwd_flag, Payment_type** are all categorical data, and mode is optimal method to replace the null values.

In [ ]:
cols = ['RatecodeID', 'store_and_fwd_flag', 'payment_type']

for col in cols:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [ ]:
data.isnull().sum()

### c. Duplicated Records

In [ ]:
data.duplicated().sum()

## 4. Outlier Analysis.

Identifying and Handling the Outliers present in the Dataset.

### a. Identifying the Outliers

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.info()

In [ ]:
# Selecting the Numerical Columns for Outlier Detection

num_cols = ['passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 
            'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 
            'congestion_surcharge']

In [ ]:
# Creating box plots for each numerical column
plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 3, i)  # Adjust rows & columns as needed
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

### b. Handling Outliers

The negative values present in the Attributes of **trip_distance, fare_amount, tip_amount, tolls_amount, total_amount** since these attributes cannot possess the neagtive values.

Hence removing them is optimal.

In [ ]:
# Convert relevant columns to numeric (forcing errors='coerce' will replace non-numeric values with NaN)
cols_to_check = ['fare_amount', 'tip_amount', 'total_amount', 'trip_distance', 'tolls_amount']
data[cols_to_check] = data[cols_to_check].apply(pd.to_numeric, errors='coerce')

# Identify rows with negative values
negative_values = data[
    (data['fare_amount'] < 0) | 
    (data['tip_amount'] < 0) | 
    (data['total_amount'] < 0) | 
    (data['trip_distance'] < 0) | 
    (data['tolls_amount'] < 0)
]

# Print the count of negative values
print("Negative value counts:")
print((data[cols_to_check] < 0).sum())

In [ ]:
# Remove rows with negative values if they are errors
data = data[
    (data['fare_amount'] >= 0) & 
    (data['tip_amount'] >= 0) & 
    (data['total_amount'] >= 0) & 
    (data['trip_distance'] >= 0) & 
    (data['tolls_amount'] >= 0)
]

In [ ]:
# Creating box plots for each numerical column
plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 3, i)  # Adjust rows & columns as needed
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

Applying the IQR(Inter Quartile Range) based filtering, Z-Score Filtering, and Log Transformations.

In [ ]:
# Function to remove outliers based on IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
from scipy import stats

# Function to remove outliers based on Z-score
def remove_outliers_zscore(df, column, threshold=3):
    z_scores = np.abs(stats.zscore(df[column], nan_policy='omit'))  # 'omit' to handle NaN values
    return df[z_scores < threshold]

In [ ]:
# Apply IQR filtering to selected columns
columns_to_filter = ['trip_distance', 'fare_amount', 'tip_amount']
for col in columns_to_filter:
    data = remove_outliers_iqr(data, col)

In [ ]:
# Apply Z-score filtering
for col in columns_to_filter:
    data = remove_outliers_zscore(data, col)

In [ ]:
# Creating box plots for each numerical column
plt.figure(figsize=(15, 10))
for i, col in enumerate(num_cols, 1):
    plt.subplot(4, 3, i)  # Adjust rows & columns as needed
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

Outliers have been significantly reduced and can be even scaled down after performing the data normalization task.

## 5. Data Normalization

In [ ]:
# Adding a small constant to avoid log(0) issues
data['trip_distance_log'] = np.log1p(data['trip_distance'])
data['fare_amount_log'] = np.log1p(data['fare_amount'])
data['tip_amount_log'] = np.log1p(data['tip_amount'])

In [ ]:
temp = ['trip_distance_log', 'fare_amount_log', 'tip_amount_log']

# Creating box plots for each numerical column
plt.figure(figsize=(15, 10))
for i, col in enumerate(temp, 1):
    plt.subplot(4, 3, i)  # Adjust rows & columns as needed
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot of {col}')

plt.tight_layout()
plt.show()

In [ ]:
data['trip_distance'] = data['trip_distance_log']
data['fare_amount'] = data['fare_amount_log']
data['tip_amount'] = data['tip_amount_log']

In [ ]:
data.describe()

## 6. Data Standardaization

In [ ]:
data.head()

In [ ]:
# Convert to datetime format (fixing the issue)
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'], errors='coerce')
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'], errors='coerce')

In [ ]:
# Splitting into separate columns for all rows
data['pickup_date'] = data['tpep_pickup_datetime'].dt.date
data['pickup_time'] = data['tpep_pickup_datetime'].dt.time

data['dropoff_date'] = data['tpep_dropoff_datetime'].dt.date
data['dropoff_time'] = data['tpep_dropoff_datetime'].dt.time

In [ ]:
data.head()

In [ ]:
data.drop({'tpep_pickup_datetime', 'tpep_dropoff_datetime'}, axis=1, inplace=True)

In [ ]:
data.drop({'trip_distance_log', 'fare_amount_log', 'tip_amount_log'}, axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data.to_csv('TaxiFareCleaned.csv', index=False)

## Data Visualization.

### 1. Bivariate Analysis Graphs

Relationship between Two Variables.

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x=data['trip_distance'], y=data['fare_amount'], alpha=0.5)
plt.xlabel("Trip Distance (miles)")
plt.ylabel("Fare Amount ($)")
plt.title("Trip Distance vs Fare Amount")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x=data['trip_distance'], y=data['total_amount'], alpha=0.5)
plt.xlabel("Trip Distance (miles)")
plt.ylabel("Total Amount ($)")
plt.title("Trip Distance vs Total Amount")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.scatterplot(x=data['trip_distance'], y=data['tip_amount'], alpha=0.5)
plt.xlabel("Trip Distance (miles)")
plt.ylabel("Tip Amount($)")
plt.title("Trip Distance vs Tip Amount")
plt.show()

### 2. Correlation Analysis.

In [ ]:
plt.figure(figsize=(13, 11))
sns.heatmap(data.select_dtypes(include=["number"]).corr(), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Feature Correlation Heatmap")
plt.show()

### 3. Time - Based Visualization

To check trip trends by hour of day.

In [ ]:
plt.figure(figsize=(10, 5))
sns.countplot(x=data['pickup_time'], palette="viridis")
plt.xlabel("Hour of the Day")
plt.ylabel("Trip Count")
plt.title("Number of Trips by Hour")
plt.show()

### 4. Interative Maps using Plotly.

In [ ]:
import plotly.express as px

fig = px.bar(data, x="payment_type", title="Trip Count by Payment Type", color="payment_type")
fig.show()